In [1]:
import sqlite3
import pandas as pd
import numpy as np
import math
from tqdm import tqdm # Optional: for progress bar
import eloModel_Functions as elo

In [2]:
# --- Main Execution ---
conn = elo.connect_db()
games_df, rp_df = elo.load_data(conn)

final_elos, pre_game_elo_df = elo.run_elo_calculation(games_df, rp_df)

Loading data from database...
Loaded 9817 games and 1420 returning production records.
Calculating Elo ratings...

Processing start of Season 2013...
Regressed ratings for 0 teams using RP metric 'usage'.

Processing start of Season 2014...
Regressed ratings for 207 teams using RP metric 'usage'.

Processing start of Season 2015...
Regressed ratings for 215 teams using RP metric 'usage'.

Processing start of Season 2016...
Regressed ratings for 223 teams using RP metric 'usage'.

Processing start of Season 2017...
Regressed ratings for 227 teams using RP metric 'usage'.

Processing start of Season 2018...
Regressed ratings for 229 teams using RP metric 'usage'.

Processing start of Season 2019...
Regressed ratings for 234 teams using RP metric 'usage'.

Processing start of Season 2020...
Regressed ratings for 235 teams using RP metric 'usage'.

Processing start of Season 2021...
Regressed ratings for 236 teams using RP metric 'usage'.

Processing start of Season 2022...
Regressed ratin

In [3]:
# --- Merge Elo back into games data ---
# Use game_id for robust merging
games_with_elo = pd.merge(games_df, pre_game_elo_df[['game_id', 'home_pregame_elo', 'away_pregame_elo']],
                            left_on='id', right_on='game_id', how='left')

# --- Basic Evaluation / Next Steps ---
# 1. Calculate Predicted Spread based on Elo
games_with_elo['predicted_spread_elo'] = games_with_elo['away_pregame_elo'] - games_with_elo['home_pregame_elo'] + elo.HFA
# Adjust for neutral site if needed (remove HFA)
games_with_elo.loc[games_with_elo['neutral_site'] == 1, 'predicted_spread_elo'] -= elo.HFA

# 2. Compare predicted_spread_elo with avg_closing_spread
# Need to drop games where closing spread is missing for fair comparison
eval_df = games_with_elo.dropna(subset=['avg_closing_spread', 'predicted_spread_elo']).copy()
if not eval_df.empty:
    mae = np.mean(np.abs(eval_df['predicted_spread_elo'] - eval_df['avg_closing_spread']))
    correlation = eval_df['predicted_spread_elo'].corr(eval_df['avg_closing_spread'])
    print(f"\nEvaluation vs Closing Spread:")
    print(f"  Mean Absolute Error (MAE): {mae:.4f}")
    print(f"  Correlation: {correlation:.4f}")

    # Check bias
    bias = np.mean(eval_df['predicted_spread_elo'] - eval_df['avg_closing_spread'])
    print(f"  Bias (Predicted - Actual): {bias:.4f}")
else:
    print("\nCould not perform evaluation: No games with both predicted Elo spread and closing spread.")

# 3. Analyze Final Ratings (Optional)
print("\nFinal Elo Ratings (Top 25):")
final_ratings_df = pd.DataFrame(list(final_elos.items()), columns=['team', 'final_elo']).sort_values('final_elo', ascending=False)
print(final_ratings_df.head(25))

# 4. Save results (Optional)
# games_with_elo.to_csv("games_with_pregame_elo.csv", index=False)
# final_ratings_df.to_csv("final_team_elos.csv", index=False)

conn.close()


Evaluation vs Closing Spread:
  Mean Absolute Error (MAE): 165.0663
  Correlation: 0.8351
  Bias (Predicted - Actual): 25.5877

Final Elo Ratings (Top 25):
               team    final_elo
134          Oregon  1831.428752
101         Georgia  1779.496702
68       Notre Dame  1770.926301
108           Texas  1767.743056
90       Ohio State  1754.255911
98       Penn State  1727.121383
115     Boise State  1714.965479
48              SMU  1705.695018
113         Alabama  1691.604862
17         Ole Miss  1690.904520
100         Clemson  1680.801697
140       Tennessee  1654.734763
30   South Carolina  1646.720340
150   Arizona State  1632.173257
132            Iowa  1618.422414
76         Michigan  1606.233608
46            Miami  1606.015455
146      Louisville  1599.959673
78         Missouri  1597.452676
111             BYU  1596.454939
139             LSU  1591.827288
66       Iowa State  1590.956168
118        Marshall  1587.602060
11             UNLV  1583.155726
44     Kansas Stat